In [ ]:
library(ggplot2)
library(dplyr)
library(hrbrthemes)
library(zoo)
library(viridis)
library(tidyr)

options(repr.plot.width = 18, repr.plot.height = 18)


In [ ]:
curr_dir <- getwd()
base_dir <- dirname(curr_dir)


base_dir

In [ ]:
# Load the DICE data from a CSV file
dice <- read.csv(file.path(base_dir, "code", "DICE.csv"))

# Replace commas with dots in the entire DataFrame
dice <- as.data.frame(lapply(dice, function(x) gsub(",", ".", x)))

# Convert all columns to numeric except 'subject'
numeric_columns <- setdiff(names(dice), 'subject')
dice[, numeric_columns] <- lapply(dice[, numeric_columns], as.numeric)

# Read the QC data from a CSV file
qc <- read.csv(file.path(base_dir, "derivatives", "QC_SC_RESULT.csv"))

# Filter rows in qc where qc_inclusion is "yes"
qc <- qc[qc$qc_inclusion == "yes", ]

# Filter rows in dice where the subject is in qc with qc_inclusion "yes"
dice <- dice[dice$subject %in% qc$subject, ]

# Select specific columns from the DICE DataFrame
selected_columns <- c('ALL', 'Frontal', 'Hippocampus', 'Frontal_Parietal',
                      'Parietal', 'Insula', 'Cingulate', 'Occipital', 'BasalGanglia',
                      'Thalamus', 'Cerebellum', 'CorpusCallosum', 'Ventricles', 'VentralDC',
                      'WM', 'Brainstem', 'Temporal')
df <- dice[selected_columns]
                             
colnames(df)[colnames(df) == "Frontal_Parietal"] <- "Paracentral Lobe"


# Reshape the DataFrame using melt function
df_melted <- reshape2::melt(df)

# Rename columns in the melted DataFrame
colnames(df_melted) <- c('ROI', 'DSC')


# Set a random color palette
set.seed(42)  # Set a seed for reproducibility
random_colors <- sample(colors(), length(unique(df_melted$ROI)))

# Create a boxplot with random colors
g1 <- ggplot(df_melted, aes(x = ROI, y = DSC, fill = ROI)) +
                             
  #geom_boxplot(color = "black", show.legend = FALSE) +
geom_boxplot(linewidth=1.2, size=8, show.legend = FALSE)+
  labs(title = '',
       x = 'ROI', y = '') +
  theme_minimal(base_size = 40) +
  theme(axis.text.x = element_text(angle = 45, hjust = 1),
        #panel.grid.major = element_blank(),
        #panel.grid.minor = element_blank(),
        panel.background = element_blank(),
        axis.line = element_line(color = 'black'))+ylim(0.125, 1)+ labs(x = NULL)+
        
        labs(x = NULL)+
        geom_hline(yintercept = 0.8, linetype = "solid", color = "black", size = 1, alpha = 0.5)+

        annotate("text", x = 1.1, y = 0.8, label = "0.8", hjust = 1, vjust = -0.5, color = "black", size = 9)


# Display the colorful boxplot
print(g1)

                             

In [ ]:
#result_df_long

In [ ]:
# Read QC, fastsurfer_vol, and freesurfer_vol data
qc <- read.csv("/home/ibrazug/Dokumente/KindersegV2/Ibra/derivatives/QC_SC_RESULT.csv")
fastsurfer_vol <- read.csv(paste0(base_dir, "/derivatives/FASTSURFER_SEG/fastsurfer_vol.csv"))
freesurfer_vol <- read.csv(paste0(base_dir, "/derivatives/FREESURFER_SEG/freesurfer_vol.csv"))

# Merge dataframes
merged_df <- merge(fastsurfer_vol, freesurfer_vol, by = 'IDs', suffixes = c('_fastsurfer', '_freesurfer'))

# Select relevant columns
relevant_cols <- colnames(fastsurfer_vol)[3:19]

# Initialize result dataframe
result_df <- data.frame(IDs = merged_df$IDs)

# Calculate percentage difference and store in result_df
for (col in relevant_cols) {
  result_df[paste0(col, "_diff")] <- ((merged_df[paste0(col, "_fastsurfer")] - merged_df[paste0(col, "_freesurfer")]) / merged_df[paste0(col, "_freesurfer")])
}

# Filter result_df based on QC inclusion
yes <- qc$subject[qc$qc_inclusion == "yes"]
result_df <- result_df[result_df$IDs %in% yes, ]

# Create a boxplot using ggplot2
result_df_long <- pivot_longer(result_df, cols = contains("_diff"),
                names_to = "Measure", values_to = "Percentage_Diff")


# Remove _diff from y-axis labels
result_df_long$Measure <- gsub("_diff", "", result_df_long$Measure)
result_df_long$Measure <- gsub("Frontal_Parietal", "Paracentral Lobe", result_df_long$Measure)
result_df_long$Measure <- gsub("CorpusCallosum", "Corpus Callosum", result_df_long$Measure)
result_df_long$Measure <- gsub("BasalGanglia", "Basal Ganglia", result_df_long$Measure)
result_df_long$Measure <- gsub("VentralDC", "Ventral DC", result_df_long$Measure)
#VentralDC
#Ventral DC
# Define the desired order of levels for Measure
desired_order <- c("eTIV", "Frontal", "Hippocampus", "Paracentral Lobe", "Parietal", "Insula", "Cingulate", "Occipital", "Basal Ganglia", "Thalamus", "Cerebellum", "Corpus Callosum", "Ventricles", "Ventral DC","WM", "Brainstem", "Temporal")

# Reorder the levels of Measure
result_df_long$Measure <- factor(result_df_long$Measure, levels = desired_order)


g2 <- ggplot(result_df_long, aes(x = Measure, y = Percentage_Diff, fill = Measure)) +
  labs(title = '', x = NULL, y = NULL) + 
  #geom_violin(width=1.5, trim=T)+
  #geom_boxplot(width=0.3)+
  geom_boxplot(linewidth=1.2, size=8)+
  labs(title = '', y = '') + 
  theme_minimal(base_size = 40) +
  theme(axis.text.x = element_text(angle = 45, hjust = 1),
        #panel.grid.major = element_blank(),
        #panel.grid.minor = element_blank(),
        panel.background = element_blank(),
        axis.line = element_line(color = 'black'), legend.position = "none")+
        ylim(-0.78, 0.78)+ 
        labs(x = NULL)+
        geom_hline(yintercept = 0, linetype = "solid", color = "black", size = 1, alpha = 0.5)


g2

In [ ]:
#both
library(gridExtra)
library(cowplot)

g1 <- g1 + labs(title = "Dice Similarity Coefficient")+ theme(plot.title = element_text(size = 40),) 
g2 <- g2 + labs(title = "Relative Volume Difference")  + theme(plot.title = element_text(size = 40),) 
#grid.arrange(g1, g2, ncol = 1)
x <- plot_grid(g1, g2, labels=c('A', 'B'), nrow=2, ncol = 1, label_size = 45,label_fontfamily = "ArialNarrow")
x